In [24]:
#
# Create model
#
import torch
import torch.nn as nn
import torch.optim as optim

class TimeSeriesTransformer(nn.Module):
    def __init__(self, d_model, nhead, num_encoder_layers, num_decoder_layers):
        super(TimeSeriesTransformer, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead),
            num_layers=num_encoder_layers
        )
        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model, nhead),
            num_layers=num_decoder_layers
        )
        self.final_layer = nn.Linear(d_model, 1)

    def forward(self, src, tgt):
        memory = self.encoder(src.permute(1, 0, 2))
        output = self.decoder(tgt.permute(1, 0, 2), memory)
        return self.final_layer(output).permute(1, 0, 2)
    
    def predict(self, src, steps_ahead=1):
        self.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # Disable gradient computation
            src = src.permute(1, 0, 2)  # Adjust dimensions
            memory = self.encoder(src)  # Encode source sequence
            tgt = torch.zeros_like(src)  # Initialize target sequence with zeros
            for i in range(steps_ahead):
                output = self.decoder(tgt, memory)  # Decode to get the next step
                tgt = torch.cat((tgt[1:], output[-1:]), dim=0)  # Append output to target sequence, drop the first time step
            predictions = self.final_layer(output)  # Apply final linear layer
        return predictions.permute(1, 0, 2)  # Adjust dimensions to match input

# Hyperparameters
d_model = 1
nhead = 1
num_encoder_layers = 6
num_decoder_layers = 6
input_dim = 1  # Because our input data has only one feature

model = TimeSeriesTransformer(d_model, nhead, num_encoder_layers, num_decoder_layers)






In [18]:
#
# Generate training data
#
import numpy as np
import torch

# Data from Berkeley 

data=[0.511811024,
0.94488189,1.771653543,2.322834646,2.834645669,3.74015748,4.05511811,3.149606299,2.047244094,1.929133858,0.62992126,0.31496063,0.62992126,0.905511811,1.417322835,2.480314961,
3.11023622,2.952755906,4.291338583,3.11023622,2.165354331,1.181102362,0.708661417,0.354330709,0.354330709,0.31496063,1.338582677,2.125984252,3.11023622,3.661417323,
4.251968504,3.385826772,1.417322835,1.417322835,0.866141732,0.275590551,0.393700787,0.94488189,0.669291339,1.968503937,3.228346457,3.858267717,3.818897638,
3.897637795,2.401574803,2.047244094,0.826771654,0.511811024,0.905511811,1.181102362,1.181102362,3.149606299,3.11023622,4.05511811,3.976377953,3.700787402,2.401574803,1.535433071,
0.472440945,0.31496063,0.905511811,1.023622047,1.692913386,2.795275591,2.913385827,3.267716535,3.937007874,3.74015748,2.677165354,1.614173228,0.62992126]

# Prepare training data
input_sequence_length = 10
output_sequence_length = 10

X = [data[i:i+input_sequence_length] for i in range(len(data)-input_sequence_length-output_sequence_length+1)]
y = [data[i+input_sequence_length:i+input_sequence_length+output_sequence_length] for i in range(len(data)-input_sequence_length-output_sequence_length+1)]

X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)  # Add dimension for features
y = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)

print(X.shape)
print(y.shape)


torch.Size([52, 10, 1])
torch.Size([52, 10, 1])


In [22]:
#
# Train model
#
import torch
import torch.optim as optim
import torch.nn as nn

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# In the training loop
output = model(X[:-1], y[1:])  # Exclude the last target value
loss = criterion(output.squeeze(-1), y[1:].squeeze(-1))  # Exclude the first target value


# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X[:-1], y[1:])  # Assuming y is the target tensor
    loss = criterion(output.squeeze(-1), y[1:].squeeze(-1))
    loss.backward()
    optimizer.step()
    
    #if epoch % 10 == 0:
    print(f'Epoch {epoch}/{num_epochs}, Loss: {loss.item()}')



Epoch 0/100, Loss: 5.222888946533203
Epoch 1/100, Loss: 5.218656063079834
Epoch 2/100, Loss: 5.214419841766357
Epoch 3/100, Loss: 5.210179805755615
Epoch 4/100, Loss: 5.205934524536133
Epoch 5/100, Loss: 5.201685428619385
Epoch 6/100, Loss: 5.1974310874938965
Epoch 7/100, Loss: 5.193171501159668
Epoch 8/100, Loss: 5.188907623291016
Epoch 9/100, Loss: 5.184637546539307
Epoch 10/100, Loss: 5.180361747741699
Epoch 11/100, Loss: 5.176080226898193
Epoch 12/100, Loss: 5.171792507171631
Epoch 13/100, Loss: 5.167500019073486
Epoch 14/100, Loss: 5.163200378417969
Epoch 15/100, Loss: 5.158894062042236
Epoch 16/100, Loss: 5.154580593109131
Epoch 17/100, Loss: 5.150261402130127
Epoch 18/100, Loss: 5.145934104919434
Epoch 19/100, Loss: 5.141600608825684
Epoch 20/100, Loss: 5.137259483337402
Epoch 21/100, Loss: 5.13291072845459
Epoch 22/100, Loss: 5.128553867340088
Epoch 23/100, Loss: 5.124190807342529
Epoch 24/100, Loss: 5.119819164276123
Epoch 25/100, Loss: 5.115438938140869
Epoch 26/100, Loss: 5.

In [31]:

# Make predictions
# Assume `new_data` is a tensor containing your new or unseen data, with dimensions [Batch Size, Sequence Length, Feature Dimension]
X_data= [data[i:i+input_sequence_length] for i in range(len(data)-input_sequence_length-output_sequence_length+1)]
new_data = torch.tensor(X_data[-1:], dtype=torch.float32).unsqueeze(-1)  # For example, using the last sequence from X_data
print(f"Shape of new data: {new_data.shape}")
steps_ahead = 10  # Assume you want to forecast 10 steps ahead
predictions = model.predict(new_data, steps_ahead)
print(f"Predictions: {predictions}")

Shape of new data: torch.Size([1, 10, 1])
Predictions: tensor([[[-0.0978],
         [-0.0978],
         [-0.0978],
         [-0.0978],
         [-0.0978],
         [-0.0978],
         [-0.0978],
         [-0.0978],
         [-0.0978],
         [-0.0978]]])
